## Description of the problem and discussion of the background
There are many cities in China, and each city has different cultures and economies. Next, I will screen the major cities in China for cluster analysis to see which cities are similar. I will use the Foursquare API to explore the most common venue categories in each city and then use this feature to group cities. I will use the k-means clustering algorithm to accomplish this task. Finally, I will use the Folium library to visualize Chinese cities and their emerging clusters.

## Data 
I have selected 35 major cities in China. Below are the names of these cities and the geographic coordinates of the city center. I will explore the most common site categories within 5 km of the city center and cluster analysis.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("China_Major_City.csv")
data

,City,Latitude,Longitude
0,Beijing,39.91667,116.41667
1,Shanghai,34.50000,121.43333
2,Tianjin,39.13333,117.20000
3,Hong Kong,22.20000,114.10000
4,Guangzhou,23.16667,113.23333
5,Shenzhen,22.61667,114.06667
6,Zhuhai,22.30000,113.51667
7,Hangzhou,30.26667,120.20000
8,Chongqing,29.56667,106.45000
9,Qingdao,36.06667,120.33333


In [3]:
data.shape

(35, 3)